In [1]:
import io, os, uuid, yaml
import numpy as np
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from src.pipeline import utils, azure_tools


In [2]:
credentials = utils.load_credentials("blob_storage")

In [3]:
container_input = "reports-ingredion"
container_client = azure_tools.get_container_client(container_input,credentials['conn_string'])

In [4]:
# name of the existing container
blob_list = azure_tools.get_blob_list(container_client)

In [5]:
container_output = "intermidiate-ingredion"

In [6]:
df = azure_tools.transform_pl_files(blob_list[:6], container_input, container_output, credentials['conn_string'])

Processing:  1-2016 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploading:  1-2016 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploded file:  US/1-2016.parquet
Processing:  1-2017 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploading:  1-2017 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploded file:  US/1-2017.parquet
Processing:  1-2018 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploading:  1-2018 US P&L by Ship To & Material(ZBW_ZANDTM001_Q00NI)
Uploded file:  US/1-2018.parquet


## Read data

In [13]:
container_client = azure_tools.get_container_client(container_output,credentials['conn_string'])
blob_list = azure_tools.get_blob_list(container_client)
blob_list

['US/1-2016.parquet', 'US/1-2017.parquet', 'US/1-2018.parquet']

In [8]:
df_test = azure_tools.read_transform_pl(container_output, credentials['conn_string'], country="US")

In [9]:
df_test.shape

(18388, 48)

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18388 entries, 0 to 5960
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Fiscal year/period                 18388 non-null  float64
 1   Company Code                       18388 non-null  object 
 2   Unnamed: 8                         18388 non-null  float64
 3   Ship to party                      18388 non-null  object 
 4   Unnamed: 10                        18388 non-null  object 
 5   Material                           18145 non-null  object 
 6   Commercial Name                    18388 non-null  object 
 7   Sales Qty Total MT                 18384 non-null  float64
 8   3P Sales Qty Total MT              16134 non-null  float64
 9   Gross Revenue USD                  18388 non-null  float64
 10  Discounts USD                      18388 non-null  float64
 11  New Net Revenue                    18388 non-null  floa